In [2]:
#Import libraries
import pandas as pd
import numpy as np
from numpy import sqrt
from numpy import vstack
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from tqdm.notebook import trange, tqdm
import time
import torch
from torch import nn
from torch.nn.functional import normalize
from torchvision import datasets, transforms
import torch.utils.data as data
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
from torch.optim import SGD
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from torch.nn.modules.activation import ReLU
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [ ]:
#Import data
DF = pd.read_csv('seattleWeather_1948-2017.csv')
DF = DF.dropna()
DF = DF.drop(['DATE'], axis=1)
DF = DF.reset_index(drop=True)
DF

In [ ]:
#Data preprocessing
#Convert categorical data to numerical data
DF['RAIN'] = DF['RAIN'].replace({'TRUE': 1, 'FALSE': 0})
DF['RAIN'] = DF['RAIN'].astype(float)

In [48]:
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.X = self.data.drop(['RAIN'], axis=1)
        self.y = self.data['RAIN']
        self.X = self.X.values
        self.y = self.y.values
        self.X = torch.from_numpy(self.X).float()
        self.y = torch.from_numpy(self.y).float()
        self.y = self.y.view(-1, 1)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [49]:
#Create the train and test dataset
train_data = Dataset(DF.iloc[:int(len(DF)*0.8)])
test_data = Dataset(DF.iloc[int(len(DF)*0.8):])

In [50]:
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
test_loader = DataLoader(test_data, batch_size=2, shuffle=True)

In [51]:
#Create the model with 4 inputs 1 hidden layer with 3 neurons and 1 output
class Model(nn.Module):
    def __init__(self,):
        super(Model, self).__init__()
        self.hidden1 = nn.Linear(4, 3)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        self.hidden2 = nn.Linear(3, 1)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = ReLU()
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        return X

In [52]:
#Define the model
model = Model().cuda()
#Define the loss function
criterion = nn.CrossEntropyLoss()
#Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [53]:
#Train the model and calculate the accuracy
def train_model(train_loader, model, loss, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))

In [54]:
train_model(train_loader, model, criterion, optimizer, epochs=5)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x3 and 4x3)